In [14]:
from operator import index

import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.stattools import medcouple
from scipy.stats import skew
import math

plt.rcParams['figure.figsize'] = [15, 7]

In [2]:
df = pd.read_csv("./outlierDatasets/synthetic.csv")
df.head()

,0,1,2,3,4,5,6,7,8,9,outlier
0,0.435518,0.038492,0.551343,0.140049,0.899545,0.588684,0.299706,0.245713,0.367375,0.452970,no
1,0.633197,0.034490,0.319406,0.879141,0.163079,0.184356,0.160583,0.104973,0.294980,0.429709,no
2,0.421558,0.299824,0.602220,0.521654,0.954621,0.547448,0.882898,0.586641,0.840204,0.212529,no
3,0.817491,0.647528,0.046214,0.487270,0.053872,0.817499,0.390589,0.394750,0.736854,0.442689,yes
4,0.291513,0.474018,0.065267,0.410573,0.903696,0.466520,0.196878,0.165370,0.297764,0.467911,no


In [25]:
X = df.iloc[:, 4:6].values
copy_X = df.iloc[:, 2:4]
y = df.iloc[:, -1:].values

(1000, 2)


In [5]:
le = LabelEncoder()
y = le.fit_transform(y)

C:\Users\khodg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
def main(X, y, n):
    nn = NearestNeighbors(n_neighbors=n)
    nn.fit(X, y)
    dist, knn = nn.kneighbors(X)  # returns N index neighbors including self
    return knn, dist


knn, distReturn = main(X, y, 10)

In [7]:
def generateArr(newDist, median='median'):
    arr = []
    for x in newDist:  # finds the distance away from that point (index 0)
        if median == 'median':
            arr += [np.median(x)]
        elif median == 'sum': # min + median + max
            arr += [x[1]+np.max(x)+np.median(x)]
        elif median == 'max':
            arr += [np.max(x)]
        elif median == 'minMax':
            arr += [np.max(x)+x[1]]
        elif median == 'medMax':
            arr += [np.median(x)+np.max(x)]
        elif median == 'min':
            arr += [x[1]]
            

    return arr

distReturn = np.array(distReturn)
arr = generateArr(distReturn, median="medMax")

# Scheme 1

In [10]:
# Standard deviation sigma = B * Q3dk

q3 = np.quantile(distReturn, 0.75)

In [56]:
def joinProb(x):
    hold = []
    mean1 = np.mean(x[:,0])
    mean2 = np.mean(x[:,1])
    print(mean1, mean2)
    sigma = q3 * 2
    for y in range(x.shape[0]):
        power = -( ( (x[:,0][y]-mean1) ** 2 + (x[:,1][y]-mean2) ** 2 ) / ((2*sigma)**2) )
        output = (1 / (2*math.pi*sigma)) * (math.e ** power)
        hold += [output]
        
    
    return hold
    
returnArr = joinProb(X)
returnArr = np.array(returnArr)
outliers = returnArr[returnArr < (max(returnArr)*.05)]
print(outliers)

0.586120019863969 0.5146356240978537
[1.63026119e-08 1.72334296e-23 3.30279556e-11 1.48202326e-30
 1.80643294e-08 4.55186623e-32 1.42245744e-03 7.69135049e-39
 6.68643531e-03 1.78618055e-08 8.50778432e-28 7.27871394e-05
 4.52063788e-07 5.68486870e-05 6.45891691e-17 1.55709485e-30
 3.17831478e-18 2.68826252e-30 3.11227242e-21 4.59357600e-17
 1.91695856e-01 1.06537688e-17 3.07266313e-18 1.83227780e-09
 1.27813209e-05 1.52144844e-38 1.19902629e-14 1.77185415e-30
 4.16522869e-14 1.35426042e-32 7.27836676e-38 1.91079932e-22
 3.64803883e-26 1.19782295e-08 2.24507620e-33 2.91684342e-16
 9.98575938e-05 1.57209157e-25 5.44432856e-11 1.01255281e-12
 1.89881526e-25 2.15916091e-26 3.54597904e-31 5.15437794e-08
 2.35437156e-08 5.21355801e-29 6.63025280e-24 7.31827487e-02
 1.18185092e-20 2.04909002e-17 8.74518624e-22 1.30744434e-14
 9.89911331e-22 2.72752513e-20 1.04058270e-30 2.81590818e-25
 3.97455396e-14 1.98636694e-29 6.39216998e-20 5.16071750e-05
 4.35162025e-04 2.02827272e-34 1.21642890e-03 1.